In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\End-to-End-Movie-Recommendation\\research'

In [2]:
import os

os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\End-to-End-Movie-Recommendation'

In [12]:
from pathlib import Path
from dataclasses import dataclass


@dataclass
class ModelTrainingConfig:
    root_dir: Path
    df_path: Path

In [13]:
from src.MoviesRecom.constants import CONFIG_PATH
from src.MoviesRecom.utils.common import read_yaml, create_directories

In [21]:
class ConfiguraitonManager:
    def __init__(self, config=CONFIG_PATH):
        self.config = read_yaml(config)

    def get_model_train_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        create_directories([config.root_dir])

        model_train_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            df_path=config.df_path
        )
        return model_train_config

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from src.MoviesRecom.logging import logger
import pandas as pd

In [28]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config = config
        self.df = pd.read_csv(self.config.df_path)
        self.similarity = None

    def bag_of_words(self):
        cv = CountVectorizer(max_features=5000, stop_words='english')
        vector = cv.fit_transform(self.df['tags']).toarray()
        self.similarity = cosine_similarity(vector)

    def recommend(self, movie):
        index = self.df[self.df['title'] == movie].index[0]
        distances = sorted(
            list(enumerate(self.similarity[index])), reverse=True, key=lambda x: x[1])
        for i in distances[1:6]:
            print(self.df.iloc[i[0]].title)

In [29]:
try:
    config = ConfiguraitonManager()
    model_train_config = config.get_model_train_config()
    model = ModelTraining(model_train_config)
    model.bag_of_words()
    model.recommend('Avatar')
except Exception as e:
    raise e

[2024-01-23 13:53:21,717: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-23 13:53:21,721: INFO: common: created directory at: artifacts/model_training]
Titan A.E.
Small Soldiers
Ender's Game
Aliens vs Predator: Requiem
Independence Day
